** Problem 1 **: *Censored data fitting* (2 points). 

<img src="1.png" / width="60%">

__Solution__

In [175]:
# data for censored fitting problem
import numpy as np
from numpy.random import randn

n = 2;  # dimension of x's
M = 25;  # number of non-censored data points
K = 100; # total number of points

c_true = randn(n)
X = randn(n, K)
y = np.dot(X.T, c_true) + 0.1 * np.sqrt(n) * randn(K)

# Reorder measurements, then censor
sort_ind = np.argsort(y)
X = X[:, sort_ind];
y = y[sort_ind[:M + 1]]
D = (y[M - 1]+y[M]) / 2
y = y[:M]

print X.shape, y.shape, c_true.shape

(2, 100) (25,) (2,)


The difference between this problem and a classical least squares is that some of the equations $c_1x_1^i + c_2x_2^i$ should now be at least $D$. We will introduce $K-M$ values $\Delta y_i$ and will try to fit the model where the error for censored values will be $(c_1x_1^i + c_2x_2^i - D - \Delta y_i)^2, i > M, \Delta y_i >= 0$, while for known $y_i (i \leq M)$ residuals will remain the same $(c_1x_1^i + c_2x_2^i - y_i)^2$

In [176]:
# Solution for part a
from cvxpy import *

c = Variable(2, 1)
delta_y = Variable(K - M)
tail = D*np.ones(K - M)
yn = np.hstack((y, tail))

obj = Minimize(norm(X.T*c-yn-vstack(np.zeros(M), delta_y)))
constraints = [ delta_y >= 0 ]
prob = Problem(obj, constraints)
res = prob.solve(solver=SCS)

print "res: ",res
print "c estimated: ", c.value.flatten(), "c true:", c_true

res:  0.570429599407
c estimated:  [[-1.72002627  0.3523026 ]] c true: [-1.70209409  0.38722166]


Now lets fit the model using only the uncensored data and then calculate the relative errors.

In [178]:
c2 = Variable(2, 1)

obj = Minimize(norm(X[:, :M].T*c2-y))
prob = Problem(obj, [])
res = prob.solve(solver=SCS)

print "res: ",res
print "c estimated: ", c2.value.flatten(), "c true:", c_true

print "Relative error case #1:", np.linalg.norm(c_true - c.value) / np.linalg.norm(c_true)
print "Relative error case #2:", np.linalg.norm(c_true - c2.value) / np.linalg.norm(c_true)

res:  0.570096582518
c estimated:  [[-1.71984295  0.35284394]] c true: [-1.70209409  0.38722166]
Relative error case #1: 1.68609790502
Relative error case #2: 1.68623492048


** Problem 2 **: *Optimal vehicle speed scheduling (3 points) *-- (additional exercise A3.20 to Boyd and Vandenberghe). 

A vehicle (say, an airplane) travels along a fixed path of *n* segments, between *n + 1* waypoints labeled *0, . . . , n*. Segment *i* starts at waypoint *i − 1* and terminates at waypoint *i*. The vehicle starts at time *t = 0* at waypoint *0*. It travels over each segment at a constant (nonnegative) speed; *si* is the speed on segment *i*. We have lower and upper limits on the speeds: *smin ≤ s ≤ smax*. The vehicle does not stop at the waypoints; it simply proceeds to the next segment. The travel distance of segment *i* is *di* (which is positive), so the travel time over segment *i* is *di/si*. We let *τi*, *i = 1, . . . , n,* denote the time at which the vehicle arrives at waypoint *i*. The vehicle is required to arrive at waypoint *i*, *for i = 1, . . . , n*, between times *τmin,i* and *τmax,i* , which are given. The vehicle consumes fuel over segment *i* at a rate that depends on its speed *Φ(s_i )=a s_i^2+b s_i+c kg/s*.

You are given the data *d* (segment travel distances), *smin* and *smax* (speed bounds), *τmin* and *τmax* (waypoint arrival time bounds), and the the parameters *a*, *b*, and *c* (all parameters are in *veh_speed_sched_data.m*). For the given form of the potentials, find the way to reduce the problem to a convex optimization problem and solve it using CVX (NB: you need not necessarily use one of the “canonical” convex optimization formulations we saw in the course). Use MATLAB command stairs to plot speed vs time for the optimal schedule.  What are relative pros and cons for using convex optimization vs. dynamic programming for such task?

__Solution__

In [232]:
import scipy.io as sio

mat_contents = sio.loadmat('veh_sched_data.mat')

n = np.int(mat_contents['n'])
a = np.float(mat_contents['a'])
b = np.float(mat_contents['b'])
c = np.float(mat_contents['c'])
d = np.array(mat_contents['d'], dtype=np.float)
smin = np.array(mat_contents['smin'], dtype=np.float)
smax = np.array(mat_contents['smax'], dtype=np.float)
tau_min = np.array(mat_contents['tau_min'], dtype=np.float)
tau_max = np.array(mat_contents['tau_max'], dtype=np.float)

Priveously we have solved this problem using DP by discretizing on each time interval. We can now reformulate this problem as CVX. 

Objective function:

$$f_0 = \min_{s}\sum\limits_{i=1}^n (as_i^2 + bs_i + c) = \min_{s}as^Ts+b\Vert s\Vert_1+cn$$

s.t.: 
$$s_i \geq 0, i \in [1 \dots n]$$
$$smin_i \leq s_i \leq smax_i$$
$$tmin_k \leq \sum\limits_{i = 1}^k \frac{d_i}{s_i} \leq tmax_k, k \in [1 \dots n] $$

In [239]:
from cvxpy import *
import numpy as np

t = Variable(n)

obj_expr = (a*d[i]**2*inv_pos(t[0]) + b*d[i] + c*t[0])

for i in xrange(1, n):
    obj_expr += (a*d[i]**2*inv_pos(t[i] - t[i - 1]) + b*d[i] + c*(t[i] - t[i - 1]))

obj = Minimize(obj_expr)

constraints = [(tau_min[i] <= t[i]) for i in xrange(n)]
constraints += [(tau_max[i] >= t[i]) for i in xrange(n)]

for i in xrange(n - 1):
    expr = smin[i]*t[i] 
    
    if i > 0:
        expr = (t[i] - t[i - 1])*smin[i]
        
    constraints += [expr <= d[i]]
    
    expr = smax[i]*t[i] 
    
    if i > 0:
        expr = (t[i] - t[i - 1])*smax[i]
        
    constraints += [expr >= d[i]]
        

prob = Problem(obj, constraints)
res = prob.solve(solver=SCS)
print "res: ",res


res:  2608.1489648


N.B.: in the first Assignment we ended up with $\approx 2617$

<img src="3.png" width="65%"/>

__Solution__

Here is Lagrangian:
$$ L(x, \lambda, \nu) = p^Tx + \lambda^T(Ax - b) + \nu^T(d - Cx)$$

$$ g(\lambda, \nu) = \min_x{L(x, \lambda, \nu)} = min_x{\left((p^T + \lambda^TA + \nu^TC)x - \lambda^Tb + \nu^Td\right)}$$

And here is dual problem formulation:

$$\max_{\nu, \lambda}{-\lambda^Tb + \nu^Td}$$
$$s.t.: p^T + \lambda^TA + \nu^TC = 0$$
$$\lambda \geq 0$$

<img src="4.png" width="65%"/>

__Solution__

$$L(x, \lambda, \nu) = f_0(x) + \lambda_1(-x_1 - 2x_2 + 1) + \lambda_2(-3x_1 - x_2 + 1) = x_1^2 + x_2^2 - x_1x_2+ \lambda_1(-x_1 - 2x_2 + 1) + \lambda_2(-3x_1 - x_2 + 1) $$

$$g(\lambda) = \min_{x}{L(x, \lambda)}$$

$$\lambda_i \geq 0, x = (x_1, x_2)^T$$

Dual problem:

$$\max_{\lambda \geq 0}g(\lambda)$$

Let's improve our $L$ representation. First let's observe that $L$ is continuous in every point on all the directions as well as infinetely differentiable. Let's find partial derevatives of $L$ w.r.t $x_1$ and $x_2$:

$$\frac{\partial L}{\partial x_1} = 2x_1-x_2-\lambda_1-3\lambda_2$$
$$\frac{\partial L}{\partial x_2} = 2x_2-x_1-2\lambda_1-\lambda_2$$

since we are minimizing a convex function the following equations should satisfy:

$$\left\{ \begin{array}{rl}
 2x_1-x_2-\lambda_1-3\lambda_2 = 0 \\
 2x_2-x_1-2\lambda_1-\lambda_2 = 0
       \end{array} \right.$$
       
lets now substitute $x_{1,2}$ in our Lagrangian:

<img src="wolfram.png" / width="40%">

$$L(x, \lambda, \nu) = -\frac{1}{3}\left(7t_1^2+17t_2t_1+13t_2^2\right) + t_1 + t_2, $$
where $t_i = \lambda_i$. 

$$L(x,\lambda,\nu) = -\frac{1}{3}t^T\left( \begin{array}{ccc}
7 & 8.5 \\
8.5 & 13  \end{array} \right)t + t_1 + t_2$$

In [60]:
import cvxpy as cx
from cvxpy import *
import numpy as np

# Initial problem
x = Variable(2, 1)
Q = np.array([1, -0.5, -0.5, 1]).reshape(2, 2)
A = np.array([1, 2, 3, 1]).reshape(2, 2)

obj = Minimize(quad_form(x, Q))
constraints = [ A*x >= 1 ]
prob = Problem(obj, constraints)
res = prob.solve(solver=SCS)
print "res: ",res
print "x: ", x.value.flatten()

# Dual problem
l = Variable(2, 1)
Q2 = np.array([7, 8.5, 8.5, 13]).reshape(2, 2)
dual_obj = Maximize(-1/3.*quad_form(l, Q2) + l[0] + l[1])
constr = [l >= 0]
prob = Problem(dual_obj, constr)
res = prob.solve(solver=SCS)
print "res dual: ",res
print "lambdas: ", l.value.flatten()

res:  0.107141954258
x:  [[ 0.28571457  0.35714399]]
res dual:  0.107140193577
lambdas:  [[  2.14091385e-01  -1.08233580e-05]]


Strong duality holds, and thus the following KKT conditions are satisfied:

$$\left\{ \begin{array}{rl}
 Ax^* - 1>=0 \\
 \lambda_{1,2} >= 0 \\
 \frac{\partial L}{\partial x}\left(x^*\right) = 0 -* 
       \end{array} \right.$$
       
\* - holds because of the equations on the partial derivatives above

Lets calculate $f_i^*(x^*)$

In [62]:
print A.dot(x.value) - 1 # greater than 0

[[  2.55066088e-06]
 [  2.14287698e-01]]


From the values above (`lambdas` and $f_i^*(x^*)$)  values we can see that the last KKT condition also holds: $\lambda_i^*f_i^*(x^*) = 0$

<img src="5_a.png" width="65%"/>

<img src="5_b.png" width="65%"/>

__Solution__

a) Here is the way we can partialy dualize (the most straightforward approach I believe) $Ax \leq b$:
$$P\left( \left(\begin{array}{ccc}
A_1 \\
0  \end{array}\right) + \left(\begin{array}{ccc}
0 \\
A_2  \end{array}\right) \right)x \leq b$$

Here $P$ is a row permutation matrix, $A_1$ is our constraints for the Lagrangian and $A_2$ are those we left untouched.

$$L(x, \lambda) = c^Tx + \lambda^TP\left(\begin{array}{ccc}
A_1 \\
0  \end{array}\right)x$$
Partial dual function:
$$g(\lambda)=\min_{x \in D' } L(x, \lambda)$$

And the $D'$ domain here contain those $x'$ that satisfy:
$$P\left(\begin{array}{ccc}
0 \\
A_2  \end{array}\right)x' \leq b$$